## NL Wallet App Simulator
- Execute each cell with Crtl+Enter
- Two cells will ask for an input. Make sure the respective QR (init, DCC) is visible on scree next to this browser window


Install libraries if you run this notebook for the first time

In [1]:
!pip install pyjwt cryptography protobuf pycryptodome ecdsa


     |████████████████████████████████| 3.6 MB 23.9 MB/s 
     |████████████████████████████████| 2.0 MB 40.7 MB/s 
     |████████████████████████████████| 119 kB 75.0 MB/s 


### Ingest an invitation

Identify yourself at an airline (e.g. https://pinggg.mywire.org/static-v2) to obtain an invitation for validation. 

Grab the invitation string from url to wallet and paste in input-box. <br>
Or alternative: Open QR-code in Google-lens and "copy text" to capture the invitation string.

In [4]:
import requests
import json
import jwt
import base64

invite = input('Paste Invitation string contents here: ')

base64_message = invite
base64_bytes = base64_message.encode('ascii')
message_bytes = base64.b64decode(base64_bytes)
qr_code = message_bytes.decode('ascii')

qr_code_data =  json.loads(qr_code)
print(f"QR-Code data: {json.dumps(qr_code_data, indent=4)}")

token_info = jwt.decode(qr_code_data['token'], options={"verify_signature":False})
print(f"Invitation-token: {token_info}")


Paste Invitation string contents here: eyJwcm90b2NvbCI6IkRDQ1ZBTElEQVRJT04iLCJwcm90b2NvbFZlcnNpb24iOiIxLjQuMCIsInNlcnZpY2VJZGVudGl0eSI6Imh0dHBzOi8vcGluZ2dnLm15d2lyZS5vcmcvd2FsbGV0L2lkZW50aXR5L3YyIiwicHJpdmFjeVVybCI6Imh0dHBzOi8vc29tZS5pbnN0aXR1dGlvbi92YWxpZGF0aW9uL3ByaXZhY3kiLCJ0b2tlbiI6ImV5SmhiR2NpT2lKRlV6STFOaUlzSW5SNWNDSTZJa3BYVkNJc0ltdHBaQ0k2SWtGd1RYaHlZMGxJTURkVFZWWlRURVo0ZGs1cVVISjJNRTg1ZEVoUVdqaDZkRmszTURkQlIxRTJVM2NpZlEuZXlKcGMzTWlPaUpvZEhSd2N6b3ZMM0JwYm1kblp5NXRlWGRwY21VdWIzSm5MM2RoYkd4bGRDOXBaR1Z1ZEdsMGVTOTJNaUlzSW5OMVlpSTZJbVUwWVRnNVpUZGhMVGRoTm1NdE5HVXhPUzFoTldZM0xXSTVaalJrWXpWbFlUUTRPQ0lzSW1saGRDSTZNVFkwTmpnME1EVXdNeXdpWlhod0lqb3hOalEyT0RRME1UQXpmUS5zRlJQVHc3TC1HTTg3TTZEVkc5bVhQY1A5V3FxekRhbkVSRkVhVnR1ejVFVUJQRjdYYVFPeHBTaDhDTExCQWpja3ZiUHczUVhKSUYtaFNFTEJzNmtYZyIsImNvbnNlbnQiOiJQbGVhc2UgY29uZmlybSB0byBzdGFydCB0aGUgRENDIEV4Y2hhbmdlIGZsb3cuIElmIHlvdSBub3QgY29uZmlybSwgdGhlIGZsb3cgaXMgYWJvcnRlZC4iLCJzdWJqZWN0IjoiZTRhODllN2EtN2E2Yy00ZTE5LWE1ZjctYjlmNGRjNWVhNDg4Iiwic2VydmljZVByb

### Init QR code handling by wallet app
The wallet app processes the invitation:


1.   The wallet gets airline identity document to learn airline endpoints and validation service location
2.   The wallet determines location of validation service ((from airline identity document)). A choice is presented in case multiple validation services are offered.

In [5]:
import ecdsa
import base64
from Crypto.Hash import SHA256
from ecdsa.curves import NIST256p

# Load the airline identity document 
serviceIdentity = requests.get( qr_code_data['serviceIdentity'] ).json()

# Get the information from the identity document. Identity contents
## services, i.e. airline endpoint to get validation access token and to return confirmation token, and (list of) providers that offer validation
## verification methods, public keys to validate signatures or for use in encryption 

validationlist=[];
for service in serviceIdentity['service']:
    
    # This should not always be 'ValidationService-1' but the current service
    # depending on rules and requirements
    if service['id'].find('#ValidationService')>0: 
        validationlist.append(service);
        validation_service_id = service['id']        
        print(f"Validation ServiceID {validation_service_id}") 
#print(validationlist)
if len(validationlist) == 1:
  choice = 0
else:
  choice = int(input('Please enter number of service you have chosen (1,2,3): ').strip())  # This picks up the validation service
  choice=choice-1
#Make record of validation service endpoint for future use
validation_service_id=validationlist[choice]['id']
validation_service_endpoint = validationlist[choice]['serviceEndpoint'] 
print(validation_service_endpoint) 

Validation ServiceID https://pinggg.mywire.org/wallet/identity/v2#ValidationService-1
http://83.84.212.40:10000


#Wallet reads the airline identity document
to get validation access token (which packs travel data and some instructions for type of validation that is expected)

The request is authenticated with invitation-token (so links back to booking and passenger)

The wallet advises its one-time identity, not sure why??

The airline hands back validation access token and a nonce (nonce, why??)

In [6]:
import ecdsa
import base64
from Crypto.Hash import SHA256
from ecdsa.curves import NIST256p

#create one-time wallet identity(key-pair) to sign the DCC on upload
userkey = ecdsa.SigningKey.generate(curve=NIST256p,hashfunc=SHA256.new) 

# App selects the AccessTokenService in Airline Identity document and requests for Validation Access token. 

for service in serviceIdentity['service']:
    if service['id'].endswith('#AccessTokenService-1'):
        response = requests.post( service['serviceEndpoint'], 
                    headers={'Authorization': f'Bearer {qr_code_data["token"]}', #initR-token goes here
                             'Content-Type':'application/json', 
                             'X-Version' : '1.0'},
                    json = {"pubKey": userkey.get_verifying_key().to_pem().decode(), 
                                    #advise pub key to validation service for inspection of signature on DCC
                            "service":validation_service_id}
                                     #advise chosen validation service
        )
        # print ("Undecoded Response", response.status_code, response.text, '\n\n')
        
        # This is the access token for the validation service
        validator_token = response.text 
        validation_nonce = response.headers['x-nonce']
        
        token_info = jwt.decode(validator_token, options={"verify_signature":False})
        print(f"Validator_Access_Token: {json.dumps(token_info, indent=4)}")




Validator_Access_Token: {
    "iss": "https://pinggg.mywire.org/wallet/identity/v2",
    "sub": "e4a89e7a-7a6c-4e19-a5f7-b9f4dc5ea488",
    "aud": "http://83.84.212.40:10000/validate",
    "t": 2,
    "v": "1.0",
    "vc": {
        "lang": "en-en",
        "coa": "NL",
        "cod": "BE",
        "poa": "AMS",
        "pod": "BRU",
        "roa": "NL",
        "rod": "BE",
        "type": [
            "r",
            "v",
            "t"
        ],
        "category": [
            "Standard"
        ],
        "validationClock": "2022-03-09T15:42:37.849Z",
        "validFrom": "2022-03-10T15:41:43.729Z",
        "validTo": "2022-03-10T16:41:43.729Z"
    },
    "jti": "d15c1cdb-6c91-4d23-8112-b23a01edb0a6",
    "iat": 1646840557,
    "exp": 1646844157
}


### Get the public key from the validation service
With the validation service endpoint that has been extracted from the decorator's identity document, we now load the validation service's identity document too to obtain its public key for encryption. 

In [7]:
validation_identity = requests.get(validation_service_endpoint+'/identity').json()

for verificationMethod in validation_identity['verificationMethod']:
    if verificationMethod['id'].endswith('ValidationServiceEncKey-1'):
        validation_service_publickey =  verificationMethod['publicKeyJwk']
        print("Selected Public Key:", validation_service_publickey )

KeyboardInterrupt: ignored

# Submit the health certificate
You can enter the health certificate data into an input field. 

Source for DCC: https://eu-dcc-validation.web.app/

Open the QR-code with Google lens (right click option in Chrome) and "copy text" to obtain health sertificate in string format 

Make sure that the name and date-of-birth match the data in the validation access token from above. 

##The wallet app now executes the following steps: 
- choose a random password (32 bytes)
- AES-encrypt the DCC with the password from above and the nonce that was obtained together with the access token 
- encrypt the password with the validation service's public key 
- sign the encrypted AES-encrypted data with the userkey that has been submitted to get the access token
- send JSON data to the validation service:
   - kid of validation server's public key that was used to encrypt password
   - encrypted dcc data
   - signature of encrypted dcc data
   - PKI encrypted password for dcc
   - constants: encScheme = RSAOAEPWithSHA256AESCBC, sigAlg = SHA256withECDSA
   - header: Access token that was previously obtained
- decode the response and print it
  - Private part of result is feedback on evaluation of individual busness rules
  - Public part of result is confirmaion token, which can be made availabe to airline if passenger wishes to do so



In [ ]:
hcert = input('Please paste your HCert: ').strip()  # Option 1: Enter the HC1:-String into the input field
assert hcert.startswith('HC1:')

from base64 import b64decode
from random import randint
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from ecdsa.util import sigencode_der

# Use real random numbers for production instead!
#password = bytearray([randint(0,255) for i in range(32)])

#validatorkey = RSA.import_key(f'-----BEGIN CERTIFICATE-----\n{validation_service_publickey["x5c"]}\n-----END CERTIFICATE-----')
#aesCipher = AES.new(password, AES.MODE_CBC,iv=b64decode(validation_nonce))

#cipher = PKCS1_OAEP.new(validatorkey,hashAlgo=SHA256)
#cryptKey = cipher.encrypt(password)
#ciphertext= aesCipher.encrypt(pad(bytes(hcert,'utf-8'),AES.block_size))
#signature = userkey.sign(ciphertext,hashfunc=SHA256.new,sigencode=sigencode_der)

#######   initialize  ###### 
headers = {'content-type': 'application/json', "Authorization":"Bearer " + validator_token ,"X-Version":"1.0"}



####### validate   ######
headers = {'content-type': 'application/json', "Authorization":"Bearer " + validator_token ,"X-Version":"1.0"}

#body = {"kid":validation_service_publickey["kid"],
#        "dcc":base64.b64encode(ciphertext).decode(),
#        "sig":base64.b64encode(signature).decode(),
#        "encKey":base64.b64encode(cryptKey).decode(),
#        "encScheme":"RSAOAEPWithSHA256AESCBC", 
#        "sigAlg":"SHA256withECDSA"}
body= {
    
}
response = requests.post(token_info['aud'], data=json.dumps(body), headers=headers)
print ("Undecoded Response", response.status_code, response.text, '\n\n')

#validation-stub#
if response.ok:
  validate_result = jwt.decode(response.content, options={"verify_signature":False})
#validation-stub#    
print(f'Validate result message: {json.dumps(validate_result, indent=4)}')
#print(f'stub, stub, stub here comes the confirmation token ')

Please paste your HCert: HC1:6BFOXN%25TSMAHN-HOO4M/MG7HG7HDX9:D43:O-36LOVAOMQV4TP5I9CQL9%2B9D*XD/GPWBILC9FF9+RPR-SPGA.N8E6KZ1WJ9OF00SI1V:4CP6C324RC2%25K:XF03LP%2BPOF4%240AQCQYCIW49W3GZS0N1B%2B4K77II*N/ME.ZJ1NGC+QAT0PI0QQ9LV7U*V:%2BGWUSN*O0%25KL%24S8J4%2BJ46YBAN8ORC4126NA6H0554-G97E9M9IDB9M%255N11NJPLEP9XA7KQSF6HX6BGUZ%2BL..P9PPTW5G0A%2BE93ZM%2496PZ6%2BQ6X46%25E5EPPQF67460R6646O59WC5OFU946%2B967KQ1C9AKPCPP7ZM-O6QW6:%250GIII7JSTNB957/5%2577P1R6ZQZ.QR95.16+K5+96P/5CA7T5M7%2B5:ZJ83BO8TGVTCET6NJF0JEYI1DLZZL162J863:QVANSMNV4G-+GNSDO5KZDP6UCI2RA9UQYH+6OYCN%25MR*Y883E5%25F6GWFSIUNT7.2D%24QE-RUSERNT/21/0FL6O-1SI59BUR%2548B40HAQW2
Undecoded Response 200 eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IiJ9.eyJpc3MiOiJodHBzOi8vc2VydmljZXByb3ZpZGVyIiwiY2F0ZWdvcnkiOiJbU3RhbmRhcmRdIiwic3ViIjoiOTFmMmQ2NWQtMjU1NS00YWI0LThhM2EtNjc5NWRmZDAzNzU2IiwicmVzdWx0IjoiT0siLCJyZXN1bHRzIjoiW10iLCJjb25maXJtYXRpb24iOiJleUpoYkdjaU9pSkZVekkxTmlJc0luUjVjQ0k2SWtwWFZDSXNJbXRwWkNJNklpSjkuZXlKcWRHa2lPaUl5WldRMk9Ea3laQzAzWWpjekx

#completion, push confirmation
Wallet offers to push confirmation to airline, as final step. Under consent.

On "yes", wallet identifies endpoint from airline identity document and POSTs the confirmation token under authentication with invitation token. In final message traeler is advised to go back to airline site

In [ ]:
ahum = input('Enter yes to push token to arline: ').strip()
  # App selects the ConfirmationService in Airline Identity document and posts the confirmation token.
for service in serviceIdentity['service']:
    if service['id'].endswith('#ConfirmationService-1'):
      response = requests.post( service['serviceEndpoint'],
                               headers={'Authorization': f'Bearer {qr_code_data["token"]}', #initR-token goes here
                               'Content-Type':'application/json', 'X-Version' : '1.0'},
                               json = {"confirmation": validate_result["confirmation"]})
print ("Undecoded Response", response.status_code, response.text, '\n\n')
        
print('Please navigate back to airline website')

Enter yes to push token to arline: 
Undecoded Response 200  


Please navigate back to airline website
